In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from utils.poetry import *
from utils.base import *
from utils.transformer_tools import *

from tqdm import tqdm

import youtokentome as yttm
import mauve

C:\Users\Ранагон\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Предобработка изначального файла, кодировка, уничтожение пустых строк

with open("datasets/Eragon/Eragon_4_cut.txt", encoding="utf-8") as input_file:
    horus = input_file.read().split('\n')
    horus = [value for value in horus if value != ""]
    # horus = [value.replace(u'\xa0', u' ') for value in horus]

with open("datasets/Eragon/Eragon4mauve.txt", "w", encoding="utf-8") as f:
    for line in horus:
        f.write(f"{line}\n")

In [8]:
for_mauve_test = load_chunks("./datasets/Eragon/Eragon4mauve.txt", chunk_size=300)


print(len(for_mauve_test))
print(for_mauve_test[23])

151
ugh. Two of the soldiers fell as Angela buried the wool combs in their sides, driving the tines right through their hauberks. The herbalist was more than a foot shorter than some of the men, but she showed no sign of fear as she bounded among them. To the contrary, she was the picture of ferocity, w


In [22]:
BPE_MODEL_FILENAME = 'models/tokens/eragon_bpe_1000.yttm'
tokenizer = yttm.BPE(BPE_MODEL_FILENAME)

G_model = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 256,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 256,
            nhead = 16,
            dim_feedforward = 512,
            dropout = 0.1
        ),
        num_layers=5
    ),
    emb_dropout = 0.1
)

In [23]:
G_model_gan = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 256,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 256,
            nhead = 16,
            dim_feedforward = 512,
            dropout = 0.1
        ),
        num_layers=5
    ),
    emb_dropout = 0.1
)

In [24]:
G_model_gan.load_state_dict(torch.load('./models/GAN/Eragon/model_G.pth'))
G_model_gan.cuda()

G_model.load_state_dict(torch.load('./models/Lord/Eragon/Lord.pth'))
G_model.cuda()

device = "cuda"

In [25]:
loop_generator = Gen_method(G_model, tokenizer)
fake_texts = []

loop_generator_gan = Gen_method(G_model_gan, tokenizer)
fake_texts_gan = []

In [30]:
for i in tqdm(range(len(for_mauve_test))):
    fake_texts.append(loop_generator(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])
    fake_texts_gan.append(loop_generator_gan(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])

100%|██████████| 151/151 [05:02<00:00,  2.00s/it]


In [31]:
print(fake_texts[4])
print(fake_texts_gan[4])

Eragon grabbed himself and said, “Is it?” “Of course,” said Brom. “Of course, I would have a choice.” “Yes,” said Brom. “
Eragon grabbed Zar’roc, and held the sword onto his knees, then clung his hands over the sword. The sword pointed at his neck, and he he


In [32]:
save_texts_to_file(fake_texts_gan, "./datasets/Eragon/Eragon_fake_rankgen.txt")
save_texts_to_file(fake_texts, "./datasets/Eragon/Eragon_fake_rankgen_old.txt")

In [33]:
out_old = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts)
print(out_old.mauve)

out_gan = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts_gan)
print(out_gan.mauve)

Featurizing q: 100%|██████████| 155/155 [01:29<00:00,  1.73it/s]


0.17324015105647228


Featurizing q: 100%|██████████| 155/155 [01:32<00:00,  1.67it/s]


0.11811598094379097
